This Python notebook is programmed to perform the ETL process of the data staging of the project. Each code bloc is guided by a text section to explain the purpose, functionality and necessity with respect to the project progress.

**Importing of Libraries**

- `pandas`: Using the `dataframe` data structure and functions handle, manipulate and analyze large datasets.

- `os`: to interact with the operating system, such as creating directories, reading and writing files, and changing the current working directory.

- `googlemaps`: Using Google Maps API to access geospatial data and services, and in particular to find locations of certain places.

In [1]:
%pip install -U googlemaps
import pandas as pd
import os
import googlemaps

In [2]:
# Need to assign a unique API key, which authenticates requests associated with project to use Google products
# API key is stored in an environment variable --> removed for security purposes
gmaps = googlemaps.Client(key='')

Read all the value from the weather data folder and combined all the weather csv into one dataframe. This allows to consolidate the multiple `.csv` files into a single DataFrame for easier analysis and manipulation, as the data is prepared for further analysis.

In [3]:
# set the path to the directory where the data files are stored
folder_path = './Weather_data'

# List all files and directories in the above folder
files = os.listdir(folder_path)

# create a list of files that start with "US"
us_files = [file for file in files if file.startswith("US")]
print("total files", len(us_files))
# Read each CSV into a DataFrame and store them in a list

dataframes = [] # Empty list to store DataFrames
for file in us_files:
    df = pd.read_csv(f"./Weather_data/{file}")
    # Extract station name from filename (remove '.csv' part)
    station_name = file[:-4]  # This removes the last 4 characters, assuming they are '.csv'
    # Add a new column with the station name
    df['station_id'] = station_name
    dataframes.append(df)
# Concatenate all DataFrames in the list into one
weather_df = pd.concat(dataframes, ignore_index=True)
weather_df  =  weather_df.drop(columns=weather_df.columns[0], axis=1) # drop the first column)
weather_df.head()


total files 210


,Date,tmax,tmin,prcp,station_id
0,1894-01-01,60.0,41.0,0.00,USC00042863
1,1894-01-02,58.0,50.0,0.40,USC00042863
2,1894-01-03,57.0,42.0,0.00,USC00042863
3,1894-01-04,53.0,42.0,0.28,USC00042863
4,1894-01-05,50.0,38.0,0.00,USC00042863


This code bloc is used for data exploration and checking of data quality, specifically by assessing the amount of missing data. As a part of data preprocessing, this is to help inform decisions about how to handle these missing values.

In [4]:
# Print out data types for weather_df and check how many many null value there are in each column
print(weather_df.dtypes)
print("tmax",weather_df['tmax'].isna().sum(),weather_df.shape[0] )
print("tmax", (weather_df['tmax'].isna().sum())/weather_df.shape[0] *100)
print("tmin",weather_df['tmin'].isna().sum(),weather_df.shape[0] )
print("tmin", (weather_df['tmin'].isna().sum())/weather_df.shape[0] *100)
print("prcp",weather_df['prcp'].isna().sum(),weather_df.shape[0] )
print("prcp", (weather_df['prcp'].isna().sum())/weather_df.shape[0] *100)
print("total NaN",  weather_df.isna().any(axis=1).sum()/ weather_df.shape[0]*100)

Date           object
tmax          float64
tmin          float64
prcp          float64
station_id     object
dtype: object
tmax 583903 10475914
tmax 5.573766642223294
tmin 583010 10475914
tmin 5.565242326349758
prcp 525810 10475914
prcp 5.019227916533106
total NaN 6.861959729719048


Given the results acquired by the previous step, most of the missing values are around 5% of the total available data. Hence, with respect to the size of the data repositroy, the missing (null) values can be simply droppped.

In [ ]:
# drop all the NaN Values
weather_df = weather_df.dropna()

In [ ]:
# check if all the values are dropped
print("total NaN",  weather_df.isna().any(axis=1).sum())
print(weather_df.head())

total NaN 0
            Date  tmax  tmin  prcp   station_id
1096  1874-01-01  60.0  36.0  0.00  USW00003822
1097  1874-01-02  62.0  43.0  0.02  USW00003822
1098  1874-01-03  66.0  53.0  0.12  USW00003822
1099  1874-01-04  70.0  59.0  0.66  USW00003822
1100  1874-01-05  66.0  58.0  0.50  USW00003822


As correct data types and formats are essential for accurate and efficient data analysis, this code bloc converts the data to the right types and format, in this case on the `weather_df` dataframe.

In [ ]:
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df["tmax"] = weather_df['tmax'].astype(int)
weather_df["tmin"] = weather_df['tmin'].astype(int)
weather_df = weather_df.reset_index()
weather_df = weather_df.drop("index",axis=1)
# weather_df = weather_df.drop(weather_df.columns[0], axis=1)
print(weather_df[:25])
print(weather_df.dtypes)

         Date  tmax  tmin  prcp   station_id
0  1874-01-01    60    36  0.00  USW00003822
1  1874-01-02    62    43  0.02  USW00003822
2  1874-01-03    66    53  0.12  USW00003822
3  1874-01-04    70    59  0.66  USW00003822
4  1874-01-05    66    58  0.50  USW00003822
5  1874-01-06    71    60  0.22  USW00003822
6  1874-01-07    59    40  0.02  USW00003822
7  1874-01-08    51    35  0.00  USW00003822
8  1874-01-09    59    35  0.03  USW00003822
9  1874-01-10    61    45  0.02  USW00003822
10 1874-01-11    68    46  0.00  USW00003822
11 1874-01-12    56    46  0.00  USW00003822
12 1874-01-13    54    46  0.00  USW00003822
13 1874-01-14    54    45  0.05  USW00003822
14 1874-01-15    46    34  0.00  USW00003822
15 1874-01-16    41    31  0.00  USW00003822
16 1874-01-17    49    29  0.00  USW00003822
17 1874-01-18    61    38  0.01  USW00003822
18 1874-01-19    63    43  0.00  USW00003822
19 1874-01-20    71    45  0.03  USW00003822
20 1874-01-21    64    51  0.09  USW00003822
21 1874-01

Generating surrogate keys for the `weather_df` dataframe to later on be used for indexing, joining data, and other data manipulation tasks, in contrast to the usage of primary (natural) keys which are not suitable in database management and data warehousing.
This step is performed for each dataframe to be created, which to then be merged all together in accordance.

In [ ]:
# Generating surrogate keys for the weather dataframe and moving the column to the first column of the dataframe
weather_df['Surrogate Keys'] = range(1,len(weather_df)+1)
weather_df = weather_df.reindex(columns=['Surrogate Keys'] + list([c for c in weather_df.columns if c!= 'Surrogate Keys']))
# print(weather_df.reset_index())
weather_df = weather_df.reset_index()
weather_df = weather_df.drop("index",axis=1)
print(weather_df.head())

   Surrogate Keys       Date  tmax  tmin  prcp   station_id
0               1 1874-01-01    60    36  0.00  USW00003822
1               2 1874-01-02    62    43  0.02  USW00003822
2               3 1874-01-03    66    53  0.12  USW00003822
3               4 1874-01-04    70    59  0.66  USW00003822
4               5 1874-01-05    66    58  0.50  USW00003822


In [ ]:
# The method takes in the month of the given date value as an argument and returns the season
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else: # month in [12, 1, 2]
        return 'Winter'

Using the previous method, the `date_df` dataframe is to be created. In particular, it is to extract useful date-related features from the `Date` column. The new features can provide valuable insights and can be used in time series analysis, trend analysis, and predictive modeling.

In [ ]:
# Convert the date column to datetime
date_df = pd.DataFrame()
print()
# Extract year, month, and day
date_df['year'] = weather_df['Date'].dt.year
date_df['month'] = weather_df['Date'].dt.month
date_df['day'] = weather_df['Date'].dt.day
# date_df['season'] = getSeason(date_df['month'])
date_df['season'] = date_df['month'].apply(get_season)

# Combine year, month, and day to create a new date column
date_df['date'] = pd.to_datetime(date_df[['year', 'month', 'day']])

# Calculate the day of the week and assign it to a new column
date_df['day_of_week'] = date_df['date'].dt.day_name()
date_df = date_df
date_df = date_df.drop("date", axis=1)
date_df[:25]

,year,month,day,season,day_of_week
0,1874,1,1,Winter,Thursday
1,1874,1,2,Winter,Friday
2,1874,1,3,Winter,Saturday
3,1874,1,4,Winter,Sunday
4,1874,1,5,Winter,Monday
5,1874,1,6,Winter,Tuesday
6,1874,1,7,Winter,Wednesday
7,1874,1,8,Winter,Thursday
8,1874,1,9,Winter,Friday
9,1874,1,10,Winter,Saturday


In [ ]:
date_df.dtypes # check the data types of the columns in the date_df dataframe

year            int32
month           int32
day             int32
season         object
day_of_week    object
dtype: object

In [ ]:
# Generating surrogate keys for the date dataframe and moving the column to the first column of the dataframe
date_df['Surrogate Keys'] = range(1,len(date_df)+1)
date_df = date_df.reindex(columns=['Surrogate Keys'] + list([c for c in date_df.columns if c!= 'Surrogate Keys']))
date_df.head()

,Surrogate Keys,year,month,day,season,day_of_week
0,1,1874,1,1,Winter,Thursday
1,2,1874,1,2,Winter,Friday
2,3,1874,1,3,Winter,Saturday
3,4,1874,1,4,Winter,Sunday
4,5,1874,1,5,Winter,Monday


Moving onto the `station_df` dataframe, created to analyse the air quality data at the individual monitoring station level. The result of a cleaned and processed DataFrame containing station information(ID, city name, latitude, longitude, and status) to then be joined with other data based on the station ID.

In [ ]:
# create a station dataframe and add data from city.csv
city_df = pd.read_csv("./Weather_data/city_info.csv")
# print(city_df, city_df.shape[0])

print(len(weather_df['station_id'].unique())) # the number of unique station IDs in the weather_df DataFrame
# Formatting steps: conversion to datetime format, sorting, and renaming columns
city_df['Stn.stDate'] = pd.to_datetime(city_df['Stn.stDate'])
city_df['Stn.edDate'] = pd.to_datetime(city_df['Stn.edDate'])
city_df['city_name']= city_df['Name']
city_df = city_df.drop('Name', axis=1)
# print(city_df)
# Sort DataFrame by ID and end date in ascending order
city_df = city_df.sort_values(by=['ID', 'Stn.edDate'], ascending=[True, False])

# Initialize an empty list to store IDs that have been marked as active
latest_status = {}
city_df['Status'] = None

city_df['Status'] = city_df['Status'].astype(str)
# Iterate through the DataFrame to update status for each ID
for index, row in city_df.iterrows():
    if row['ID'] not in latest_status:
        latest_status[row['ID']] = 'Active' if row['Stn.edDate'] == pd.to_datetime("2021-12-31") else 'Inactive'
    city_df.loc[index, 'Status'] = latest_status[row['ID']]

# Add 'Status' column to DataFrame
# df['Status'] = statuses
    # removes duplicate rows based on 'ID' and drops any remaining rows with missing values
    city_df = city_df.drop_duplicates(subset=['ID']).dropna()

# Select only necessary columns
result_df = city_df[['city_name', 'ID','Lat','Lon','Status']].reset_index()

station_df = result_df.drop("index", axis=1)
print(station_df)

210
        city_name           ID      Lat       Lon  Status
0       Escondido  USC00042863  33.1211 -117.0900  Active
1    Natchitoches  USC00166584  31.8142  -93.0856  Active
2       Belvidere  USC00280734  40.8292  -75.0833  Active
3    NewBrunswick  USC00286055  40.4728  -74.4225  Active
4        Portland  USC00356749  45.5181 -122.6894  Active
..            ...          ...      ...       ...     ...
205      Waterloo  USW00094910  42.5544  -92.4011  Active
206       Ashland  USW00094929  46.5486  -90.9189  Active
207      Hastings  USW00094949  40.6044  -98.4272  Active
208      Mitchell  USW00094950  43.7667  -98.0333  Active
209       Hayward  USW00094973  46.0261  -91.4442  Active

[210 rows x 5 columns]


In [ ]:
# # Generating surrogate keys for the station dataframe and moving the column to the first column of the dataframe
station_df['Surrogate Keys'] = range(1,len(station_df)+1)
station_df = station_df.reindex(columns=['Surrogate Keys'] + list([c for c in station_df.columns if c!= 'Surrogate Keys']))
station_df.head()

,Surrogate Keys,city_name,ID,Lat,Lon,Status
0,1,Escondido,USC00042863,33.1211,-117.0900,Active
1,2,Natchitoches,USC00166584,31.8142,-93.0856,Active
2,3,Belvidere,USC00280734,40.8292,-75.0833,Active
3,4,NewBrunswick,USC00286055,40.4728,-74.4225,Active
4,5,Portland,USC00356749,45.5181,-122.6894,Active


The `US_AQI.csv` dataset is the main dataset, containing the measures of the FACT table to be created at the end of the ETL process. In addition, some new features are engineered to enrich the data analysis aspects.

In [ ]:
usAQi_df = pd.read_csv("US_AQI.csv")
usAQi_df.head()

,Unnamed: 0,CBSA Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_id,state_name,lat,lng,population,density,timezone
0,0,10140,2022-01-01,21,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
1,1,10140,2022-01-02,12,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
2,2,10140,2022-01-03,18,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
3,3,10140,2022-01-04,19,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
4,4,10140,2022-01-05,17,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles


Based upon the above, the `geography_df` dataframe is constructed. This way, it allows to be in accordance with the main dataset, and allow the a focused approach to the following preprocessing steps. The dataframe includes geographical attributes such as area code, city, state, population, population density, and timezone.

In [ ]:
# constructing geographic dataframe, columnized by the attributes: id, city , state, population, density, timezone
# data types of columns are also checked to ensure they are in the correct format
usAQi_df = pd.read_csv("US_AQI.csv")
print("types", usAQi_df.dtypes)
usAQi_df['Date'] = pd.to_datetime(usAQi_df['Date'])
# usAQi_df.head()

types Unnamed: 0                     int64
CBSA Code                      int64
Date                          object
AQI                            int64
Category                      object
Defining Parameter            object
Number of Sites Reporting      int64
city_ascii                    object
state_id                      object
state_name                    object
lat                          float64
lng                          float64
population                   float64
density                      float64
timezone                      object
dtype: object


In [ ]:
geography_df = pd.DataFrame()

usCites = pd.read_csv("uscities.csv")
print(len(usAQi_df['city_ascii']), len(usAQi_df['state_name']))
# add column "Date" for the geography dataframe and convert the column to datetime format
usAQi_df["Date"] = pd.to_datetime(usAQi_df["Date"])

# providing the column names for the geography dataframe
geography_df['Area Code'] = usAQi_df['CBSA Code']
geography_df["City"] = usAQi_df['city_ascii']
geography_df["State"] = usAQi_df['state_name']
geography_df["Population"] = usAQi_df['population']
geography_df["Population Density"] = usAQi_df['density']
geography_df["Timezone"] = usAQi_df['timezone']


# change type of population column to integer and check the data types of the columns in the geography_df dataframe
geography_df["Population"]= geography_df["Population"].astype(int)
print(geography_df.dtypes)

geography_df = geography_df.drop_duplicates()
geography_df.head()
#remove NaN value
# geography_df.dropna()
# print(geography_df)

5617325 5617325
Area Code               int64
City                   object
State                  object
Population              int64
Population Density    float64
Timezone               object
dtype: object


,Area Code,City,State,Population,Population Density,Timezone
0,10140,Aberdeen,Washington,16571,588.0,America/Los_Angeles
31,10420,Akron,Ohio,570375,1230.0,America/New_York
121,10500,Albany,Georgia,89323,509.0,America/New_York
209,10540,Albany,Oregon,66405,1190.0,America/Los_Angeles
299,10580,Albany,New York,590823,1747.0,America/New_York


In [ ]:
# Generating surrogate keys for the geography dataframe and moving the column to the first column of the dataframe
geography_df['Surrogate Keys'] = range(1,len(geography_df)+1)
geography_df = geography_df.reindex(columns=['Surrogate Keys'] + list([c for c in geography_df.columns if c!= 'Surrogate Keys'])).reset_index()
geography_df = geography_df.drop("index",axis=1)


In [ ]:
print(geography_df.shape[0])
geography_df.head(25)

671


,Surrogate Keys,Area Code,City,State,Population,Population Density,Timezone
0,1,10140,Aberdeen,Washington,16571,588.0,America/Los_Angeles
1,2,10420,Akron,Ohio,570375,1230.0,America/New_York
2,3,10500,Albany,Georgia,89323,509.0,America/New_York
3,4,10540,Albany,Oregon,66405,1190.0,America/Los_Angeles
4,5,10580,Albany,New York,590823,1747.0,America/New_York
5,6,10740,Albuquerque,New Mexico,762853,1155.0,America/Denver
6,7,11100,Amarillo,Texas,202902,751.0,America/Chicago
7,8,11140,Americus,Georgia,15319,521.0,America/New_York
8,9,11460,Ann Arbor,Michigan,323593,1657.0,America/Detroit
9,10,11540,Appleton,Wisconsin,225728,1150.0,America/Chicago


To construct the `pollutant_df` dataframe, the poulltant datasets each need to be processed and analysed thoroughly. This is to establish a universal convention for all pollutants within the data mart, with each providing data based on the same ID (in this case, all can be linked via the `Date` attribute).

In [ ]:
folder_path = './NO2'

# List all files and directories in the folder
files = os.listdir(folder_path) # folder path for NO2
files_co = os.listdir("./CO")
files_so2 = os.listdir("./SO2")
files_o3 = os.listdir("./O3")
files_pm2_5 = os.listdir("./pm2.5")
files_pm10 = os.listdir("./pm10")

print("files length(should be same):",len(files),len(files_co),len(files_so2))
files.remove(".DS_Store")
# files_co.remove(".DS_Store")
dataframes = []
dataframes_co=[]
dataframes_so2 =[]
dataframes_o3 =[]
dataframes_pm2_5=[]
dataframes_pm10=[]

# Read each CSV into a DataFrame and store and append them in a list
for i in range(len(files)):
    df = pd.read_csv(f"./NO2/{files[i]}",low_memory=False)
    df2 = pd.read_csv(f"./CO/{files_co[i]}",low_memory=False)
    df3 = pd.read_csv((f"./SO2/{files_so2[i]}"),low_memory=False)
    df4 = pd.read_csv((f"./O3/{files_o3[i]}"),low_memory=False)
    df5 = pd.read_csv((f"./pm2.5//{files_pm2_5[i]}"),low_memory=False)
    df6 = pd.read_csv((f"./pm10/{files_pm10[i]}"),low_memory=False)

    # Extract station name from filename (remove '.csv' part)
    # station_name = file[:-4]  # This removes the last 4 characters, assuming they are '.csv'
    # Add a new column with the station name
    # df['station_id'] = station_name
    dataframes.append(df)
    dataframes_co.append(df2)
    dataframes_so2.append(df3)
    dataframes_o3.append(df4)
    dataframes_pm2_5.append(df5)
    dataframes_pm10.append(df6)

no2_df = pd.concat(dataframes, ignore_index=True)
co_df = pd.concat(dataframes_co, ignore_index=True)
so2_df = pd.concat(dataframes_so2, ignore_index=True)
o3_df = pd.concat(dataframes_o3, ignore_index=True)
pm2_5_df = pd.concat(dataframes_pm2_5, ignore_index=True)
pm10_df = pd.concat(dataframes_pm10, ignore_index=True)

# Concatenate all DataFrames in the list into a single dataframe

files length(should be same): 43 43 43


/var/folders/1q/tg3j4nxj1r15d72yljd9qj9h0000gn/T/ipykernel_76062/2777298108.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pm2_5_df = pd.concat(dataframes_pm2_5, ignore_index=True)
/var/folders/1q/tg3j4nxj1r15d72yljd9qj9h0000gn/T/ipykernel_76062/2777298108.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pm10_df = pd.concat(dataframes_pm10, ignore_index=True)


To faciliate the operation for all pollutants, this is a helper function that takes a DataFrame and a pollutant name as input. It then creates a new DataFrame with specific columns from the input DataFrame, renames the `Arithmetic Mean` column to the name of the pollutant, converts the `Date Local` column to datetime format, and removes rows where the city is `Not in a city`. It returns the processed DataFrame.

In [ ]:
def process_pollutant_df(df, pollutant_name):
    # new_df = df[["Date Local", "Arithmetic Mean", "State Name", "Sample Duration", "City Name"]]
    new_df = pd.DataFrame()
    new_df['Date'] = df['Date Local']
    new_df['State'] = df['State Name']
    new_df['City'] = df['City Name']
    new_df["Sample Duration"] = df["Sample Duration"]
    new_df[f"{pollutant_name} Mean"] = df["Arithmetic Mean"]
    new_df['Date'] = pd.to_datetime(df['Date Local'])  # Convert 'Date Local' to datetime
    new_df = new_df[new_df['City'] != "Not in a city"]  # Drop rows where 'City Name' is "Not in a city"
    return new_df

To prepare the pollutants data for easier formatting and further analysis, the following code bloc cleans and processes the data, and also checks for missing values which can affect the accuracy of the analysis.

In [ ]:
#create a polluatant data frame and filter pollutant data
pollutant_df = pd.DataFrame()
# create a dictionary to store the pollutant dataframes: keys are the pollutant names and values are the pollutant dataframes
pollutant_dfs = {
    'NO2': no2_df,
    'CO': co_df,
    'SO2': so2_df,
    'O3': o3_df,
    'PM2.5': pm2_5_df,
    'PM10': pm10_df
}

# Process each dataframe
for pollutant, df in pollutant_dfs.items():
    print(df.head(0))
    pollutant_dfs[pollutant] = process_pollutant_df(df, pollutant)

    # Print datatype information
    print(f"{pollutant} DataFrame dtypes:")
    print(pollutant_dfs[pollutant].dtypes)

    # Check for NaN values
    print(f"{pollutant} DataFrame NaN values:")
    print(pollutant_dfs[pollutant].isna().sum())

no2_df = pollutant_dfs['NO2']
co_df = pollutant_dfs['CO']
so2_df = pollutant_dfs['SO2']
o3_df = pollutant_dfs['O3']
pm10_df = pollutant_dfs['PM10']
pm2_5_df = pollutant_dfs['PM2.5']

Empty DataFrame
Columns: [State Code, County Code, Site Num, Parameter Code, POC, Latitude, Longitude, Datum, Parameter Name, Sample Duration, Pollutant Standard, Date Local, Units of Measure, Event Type, Observation Count, Observation Percent, Arithmetic Mean, 1st Max Value, 1st Max Hour, AQI, Method Code, Method Name, Local Site Name, Address, State Name, County Name, City Name, CBSA Name, Date of Last Change]
Index: []

[0 rows x 29 columns]
NO2 DataFrame dtypes:
Date               datetime64[ns]
State                      object
City                       object
Sample Duration            object
NO2 Mean                  float64
dtype: object
NO2 DataFrame NaN values:
Date               0
State              0
City               0
Sample Duration    0
NO2 Mean           0
dtype: int64
Empty DataFrame
Columns: [State Code, County Code, Site Num, Parameter Code, POC, Latitude, Longitude, Datum, Parameter Name, Sample Duration, Pollutant Standard, Date Local, Units of Measure, Event Ty

Keep one variation of the sample duration and then drop the column `Sample Duration` as it was only needed for filteration. To be specific, the pollutant data is further cleaned by removing unnecessary rows and columns. The filtering is necessary because the analysis may require data that was sampled at specific intervals (e.g., every 1 hour, every 8 hours, or every 24 hours). The column dropping is necessary to simplify the DataFrames and reduce memory usage.

In [ ]:
# clean the repeated value measure and take the 8hour average
# print(pm10_df["Sample Duration"].unique())
# print(co_df[co_df['State Name'] == "Arizona"].shape[0])
filtered_co = co_df[co_df['Sample Duration'] == "1 HOUR"]
filtered_no2 = no2_df[no2_df['Sample Duration'] == "1 HOUR"]
filtered_so2 = so2_df[so2_df['Sample Duration'] == "1 HOUR"]
filtered_o3 = o3_df[o3_df['Sample Duration'] == "8-HR RUN AVG BEGIN HOUR"]
filtered_pm2_5 = pm2_5_df[pm2_5_df['Sample Duration'] == "1 HOUR"]
filtered_pm10 = pm10_df[pm10_df['Sample Duration'] == "24 HOUR"]


# print(filtered_o3.shape[0])
# drop sample coloum
filtered_co=filtered_co.drop("Sample Duration", axis=1)
filtered_no2=filtered_no2.drop("Sample Duration", axis=1)
filtered_so2=filtered_so2.drop("Sample Duration", axis=1)
filtered_o3=filtered_o3.drop("Sample Duration", axis=1)
filtered_pm2_5 = filtered_pm2_5.drop("Sample Duration", axis=1)
filtered_pm10=filtered_pm10.drop("Sample Duration", axis=1)

# print(filtered_co.shape[0])
# print(filtered_co.head(25))
# print(filtered_no2.shape[0])
# print(filtered_o3.shape[0])

Now joining all the pollutants using **outer join** since we want to know which row result in null. Merging all the pollutant DataFrames into a single DataFrame for easier analysis, simplifying the data structure and makings it easier to perform analysis on all the pollutants together.

In [ ]:
# join the polluant using inner join on date
# create a list of the filtered pollutant DataFrame
filtered_pollutant_list = [filtered_co, filtered_no2, filtered_so2, filtered_o3, filtered_pm2_5, filtered_pm10]

# initialize as the first DataFrame in the list
joined_final = filtered_pollutant_list[0]

# Loop through the list and merge each DataFrame to the final DataFrame
for df in filtered_pollutant_list[1:]:
    # print(df.head(0))
    joined_final = joined_final.merge(df, on=['City', 'State', 'Date'], how="outer")


In [ ]:
# Sort, reset index, and display the head of the final DataFrame
joined_final = joined_final.sort_values(by='Date')
joined_final = joined_final.reset_index()
joined_final = joined_final.drop("index", axis=1)

# Display the head of the final DataFrame
print(joined_final.head(2))

        Date         State          City   CO Mean   NO2 Mean  SO2 Mean  \
0 1980-01-01          Iowa     Davenport  0.000000        NaN       NaN   
1 1980-01-01  Pennsylvania  Philadelphia  3.833333  46.666667      17.5   

    O3 Mean  PM2.5 Mean  PM10 Mean  
0       NaN         NaN        NaN  
1  0.004294         NaN        NaN  


Since there are many oberservations done, they are grouped as `[DATE, STATE, CITY]`, then averaged by the mean on the pollutant value. All pollutant measurements are converted to be of the same units as well.

In [ ]:
print(joined_final.shape[0])
# print(joined_final.isna().sum())
print("Missing Rows for CO %",joined_final['CO Mean'].isna().sum()/joined_final.shape[0]*100)
print("Missing Rows for NO2 %",joined_final['NO2 Mean'].isna().sum()/joined_final.shape[0]*100)
print("Missing Rows for SO2 %",joined_final['SO2 Mean'].isna().sum()/joined_final.shape[0] *100)
print("Missing Rows for O3 %",joined_final['O3 Mean'].isna().sum()/joined_final.shape[0])

print('Missing Rows for PM 2.5 %',joined_final['PM2.5 Mean'].isna().sum()/joined_final.shape[0]*100)
print('Missing Rows for PM 10 %',joined_final['PM10 Mean'].isna().sum()/joined_final.shape[0]*100)


df_sorted_by_date = joined_final.sort_values('Date')
# print(df_sorted_by_date.head(10))
grouped_df = df_sorted_by_date.groupby(['City', 'State', 'Date']).agg({
    'NO2 Mean': 'mean',
    'CO Mean':'mean',
    'SO2 Mean': 'mean',
    'O3 Mean': 'mean',
    'PM2.5 Mean':'mean',
    'PM10 Mean':'mean'
    # Add more columns and their aggregation functions as needed
}).reset_index()
grouped_df = grouped_df.sort_values('Date')
# convert ppb and μg/m3 to ppm for consistent units
grouped_df['NO2 Mean'] = grouped_df['NO2 Mean'] / 1000
grouped_df['SO2 Mean'] = grouped_df['SO2 Mean'] / 1000
grouped_df['PM10 Mean'] = grouped_df['PM10 Mean']/1000
grouped_df['PM2.5 Mean'] = grouped_df['PM2.5 Mean']/1000

print(grouped_df.head(10))
print(grouped_df.shape[0])
pollutant_df = grouped_df.copy()

147091898


Missig Rows for CO % 7.663634199621246
Missig Rows for NO2 % 8.75496827160392
Missig Rows for SO2 % 7.514538292245029
Missig Rows for O3 % 0.062197450195387374
Missing Rows for PM 2.5 % 40.349170013429294
Missing Rows for PM 10 % 49.897546362478785
                      City           State       Date  NO2 Mean   CO Mean  \
9139466             Peoria        Illinois 1980-01-01       NaN  0.462500   
2647209         Costa Mesa      California 1980-01-01   0.04087  4.208333   
10101561         Rock Hill  South Carolina 1980-01-01       NaN  0.891667   
1366981            Bristol        Virginia 1980-01-01       NaN       NaN   
12455180            Warren            Ohio 1980-01-01       NaN       NaN   
4470756          Glen Cove        New York 1980-01-01       NaN       NaN   
13019964  Wisconsin Rapids       Wisconsin 1980-01-01       NaN  0.529167   
12558497          Waukesha       Wisconsin 1980-01-01       NaN  0.300000   
2431362           Columbia  South Carolina 1980-01-01     

To handle the missing values problem in the pollutant data, the **time-based interpolation** is a reasonable method for filling missing values in time series data because it assumes that the data follows a linear trend over time. As such, the accuracy of the analysis is not diminished, but rather favoured.

In [ ]:
test_df = pd.DataFrame()
test_df = pollutant_df.copy()
# print(test_df.head())
# print(pollutant_df.head())
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df.set_index('Date', inplace=True)

# Perform time-based interpolation
# interpolates missing values in each pollutant column based on time.
# The 'both' limit direction means that missing values at the beginning and end of the series are filled as well
for column in ['NO2 Mean', 'CO Mean', 'SO2 Mean', 'O3 Mean', 'PM2.5 Mean', 'PM10 Mean']:
    test_df[column].interpolate(method='time', inplace=True, limit_direction='both')
print(test_df.head(1600000))

# print the number of missing values in each column after interpolation
print(test_df.shape[0])
# print(joined_final.isna().sum())
print("Missing Rows for CO %",test_df['CO Mean'].isna().sum()/test_df.shape[0]*100)
print("Missing Rows for NO2 %",test_df['NO2 Mean'].isna().sum()/test_df.shape[0]*100)
print("Missing Rows for SO2 %",test_df['SO2 Mean'].isna().sum()/test_df.shape[0] *100)
print("Missing Rows for O3 %",test_df['O3 Mean'].isna().sum()/test_df.shape[0]*100)

print('Missing Rows for PM 2.5 %',test_df['PM2.5 Mean'].isna().sum()/test_df.shape[0]*100)
print('Missing Rows for PM 10 %',test_df['PM10 Mean'].isna().sum()/test_df.shape[0]*100)

                     City           State  NO2 Mean   CO Mean  SO2 Mean  \
Date                                                                      
1980-01-01         Peoria        Illinois  0.000000  0.462500  0.000375   
1980-01-01     Costa Mesa      California  0.040870  4.208333  0.004583   
1980-01-01      Rock Hill  South Carolina  0.000000  0.891667  0.021542   
1980-01-01        Bristol        Virginia  0.000000  3.645833  0.035417   
1980-01-01         Warren            Ohio  0.000000  3.645833  0.008696   
...                   ...             ...       ...       ...       ...   
1986-07-10  San Francisco      California  0.016458  1.500000  0.005833   
1986-07-10      Pawtucket    Rhode Island  0.015909  1.500000  0.002100   
1986-07-10      Hollister      California  0.015909  1.500000  0.003043   
1986-07-10   Granite City        Illinois  0.015909  1.333333  0.003043   
1986-07-10         Newark      New Jersey  0.015750  0.616667  0.004625   

             O3 Mean  PM

In [ ]:
# Add the Risk level to the pollutant based on the main dataset for the each date
pollutant_df = test_df
pollutant_df = pd.merge(pollutant_df, usAQi_df , left_on=['State','City','Date'],right_on=['state_name','city_ascii','Date'],how="left")

In [ ]:
# reassigning the columns of the pollutant dataframe to the desired order and display the head of the dataframe
pollutant_df = pollutant_df[['Date','State','City','NO2 Mean',"CO Mean", "SO2 Mean","O3 Mean","PM2.5 Mean","PM10 Mean","Category"]]
pollutant_df.head()

,Date,State,City,NO2 Mean,CO Mean,SO2 Mean,O3 Mean,PM2.5 Mean,PM10 Mean,Category
0,1980-01-01,Illinois,Peoria,0.00000,0.462500,0.000375,0.008706,0.015688,0.024,Good
1,1980-01-01,California,Costa Mesa,0.04087,4.208333,0.004583,0.004412,0.015688,0.024,NaN
2,1980-01-01,South Carolina,Rock Hill,0.00000,0.891667,0.021542,0.003118,0.015688,0.024,NaN
3,1980-01-01,Virginia,Bristol,0.00000,3.645833,0.035417,0.003000,0.015688,0.024,NaN
4,1980-01-01,Ohio,Warren,0.00000,3.645833,0.008696,0.003000,0.015688,0.024,NaN


In [ ]:
# Generating surrogate keys for the pollutant dataframe and moving the column to the first column of the dataframe
pollutant_df['Surrogate Keys'] = range(1,len(pollutant_df)+1)
pollutant_df = pollutant_df.reindex(columns=['Surrogate Keys'] + list([c for c in pollutant_df.columns if c!= 'Surrogate Keys']))
pollutant_df.head()

,Surrogate Keys,Date,State,City,NO2 Mean,CO Mean,SO2 Mean,O3 Mean,PM2.5 Mean,PM10 Mean,Category
0,1,1980-01-01,Illinois,Peoria,0.00000,0.462500,0.000375,0.008706,0.015688,0.024,Good
1,2,1980-01-01,California,Costa Mesa,0.04087,4.208333,0.004583,0.004412,0.015688,0.024,NaN
2,3,1980-01-01,South Carolina,Rock Hill,0.00000,0.891667,0.021542,0.003118,0.015688,0.024,NaN
3,4,1980-01-01,Virginia,Bristol,0.00000,3.645833,0.035417,0.003000,0.015688,0.024,NaN
4,5,1980-01-01,Ohio,Warren,0.00000,3.645833,0.008696,0.003000,0.015688,0.024,NaN


In [ ]:
# to write to csv
# df.to_csv(file_name, encoding='utf-8', index=False)

At this point on, the merging process begins, where a certain pair of dataframes are joined based on a common attribute. Before doing so, some further preprocessing is required, to ease the operations and perform in an effective and effecient manner. First up is to merge the pollutant and geographical data into a single DataFrame. This allows the user to analyze the pollutant data in the context of geographical information. In addition, the calculation of the number and percentage of deleted rows is useful for understanding the impact of the merge operation on the data.

In [ ]:
# stores the number of rows in pollutant_df before merging with geography_df
row_tmp_sum1 = pollutant_df.shape[0]
# merges pollutant_df and geography_df based on the 'City' and 'State' columns
# The result is a DataFrame that includes both pollutant and geographical information for each city and state.
merge_pollutant_geo = pollutant_df.merge(geography_df, left_on=['City', 'State',], right_on=['City', 'State', ])

# merge_pollutant_geo = merge_pollutant_geo.drop(["population","population_density","timezone",'City', 'State'],axis=1)
# This stores the number of rows in merge_pollutant_geo after merging with geography_df
row_tmp_sum2 = (merge_pollutant_geo.shape[0])
# print(type(row_tmp_sum1))
print(row_tmp_sum2)
print("number of rows deleted from pollutant:" ,(row_tmp_sum1 - row_tmp_sum2 ), ((row_tmp_sum1 - row_tmp_sum2)/row_tmp_sum1)*100,"%" )
print(merge_pollutant_geo.head())

3930979
number of rows deleted from pollutant: 9304793 70.30034213342448 %
   Surrogate Keys_x       Date     State    City  NO2 Mean   CO Mean  \
0                 1 1980-01-01  Illinois  Peoria       NaN  0.462500   
1               688 1980-01-02  Illinois  Peoria       NaN  0.858333   
2              1651 1980-01-03  Illinois  Peoria       NaN  0.770000   
3              1897 1980-01-04  Illinois  Peoria       NaN  1.266667   
4              2420 1980-01-05  Illinois  Peoria       NaN  1.137500   

   SO2 Mean   O3 Mean  PM2.5 Mean  PM10 Mean Category  Surrogate Keys_y  \
0  0.000375  0.008706         NaN        NaN     Good               231   
1  0.001208  0.002529         NaN        NaN     Good               231   
2  0.006987  0.004235         NaN        NaN     Good               231   
3  0.010042  0.003824         NaN        NaN     Good               231   
4  0.008063  0.008647         NaN        NaN     Good               231   

   Area Code  Population  Population Dens

In [ ]:
# print(jointest.head())
# merging on the 'City' and 'State' columns
merged_df = pd.merge(pollutant_df, geography_df, how='left', left_on=['City', 'State'], right_on=['City', 'State'])

# Filter out the rows where the join from df2 resulted in NaN values
non_matching_rows = merged_df[merged_df['City'].isnull() | merged_df['State'].isnull()]

print(non_matching_rows['State'].unique(), non_matching_rows['City'].unique())

[] []


In [ ]:
#rows that were removed when merging pollutant and geo
print(non_matching_rows.head(800))

Empty DataFrame
Columns: [Surrogate Keys_x, Date, State, City, NO2 Mean, CO Mean, SO2 Mean, O3 Mean, PM2.5 Mean, PM10 Mean, Category, Surrogate Keys_y, Area Code, Population, Population Density, Timezone]
Index: []


With the Google Maps API, this method performs reverse geocoding, which converts geographic coordinates into a human-readable address. It then extracts the `city` and `state` from the address components and returns them (latitude and longitude --> city and state).

In [ ]:
def get_city_state(lat, lon):
    reverse_geocode_result = gmaps.reverse_geocode((lat, lon))
    city, state = None, None
    for component in reverse_geocode_result[0]['address_components']:
        if 'locality' in component['types']:
            city = component['long_name']
        elif 'administrative_area_level_1' in component['types']:
            state = component['long_name']
    return city, state

# Create a wrapper function to apply on the DataFrame
def apply_get_city_state(row):
    return get_city_state(row['Lat'], row['Lon'])

In [ ]:
# Apply the above function to the Station DataFrame to get the city and state for each row
city_state_df = station_df.apply(apply_get_city_state, axis=1, result_type='expand')
city_state_df.columns = ['City', 'State']

A key note here to distinguish the different merging techniques is that `merge()` is used to combine two (or more) dataframes on the basis of values of common columns (as used via *indices*: `left_index=True` and/or `right_index=True`), and `concat()` is used to append one (or more) dataframes one below the other (or sideways, depending on whether the axis option is set to 0 or 1). `join()` is used to merge 2 dataframes on the basis of the index; instead of using `merge()` with the option `left_index=True` we can use `join()`.

In [ ]:
# Concatenate the city_state_df DataFrame with the station_df DataFrame
station_df = pd.concat([station_df, city_state_df], axis=1)
station_df = station_df.drop("city_name", axis=1)
print(station_df)

     Surrogate Keys           ID      Lat       Lon  Status            City  \
0                 1  USC00042863  33.1211 -117.0900  Active       Escondido   
1                 2  USC00166584  31.8142  -93.0856  Active    Natchitoches   
2                 3  USC00280734  40.8292  -75.0833  Active       Belvidere   
3                 4  USC00286055  40.4728  -74.4225  Active  East Brunswick   
4                 5  USC00356749  45.5181 -122.6894  Active        Portland   
..              ...          ...      ...       ...     ...             ...   
205             206  USW00094910  42.5544  -92.4011  Active        Waterloo   
206             207  USW00094929  46.5486  -90.9189  Active         Ashland   
207             208  USW00094949  40.6044  -98.4272  Active        Hastings   
208             209  USW00094950  43.7667  -98.0333  Active        Mitchell   
209             210  USW00094973  46.0261  -91.4442  Active         Hayward   

            State  
0      California  
1       Lou

This new dataframe regarding `date_main` is to extract the granular components of the main dataset, so that all other dataframes are to be in sync with respect to these `date` attribute components (since the measures are availbale for a certain period of time ranges).

In [ ]:
# filter by dates

# Extract and sort the record based on assencding
date_main = pd.DataFrame()
# Extract year, month, and day
date_main['day'] = usAQi_df['Date'].dt.day
date_main['month'] = usAQi_df['Date'].dt.month

date_main['year'] = usAQi_df['Date'].dt.year

# date_df['season'] = getSeason(date_df['month'])

date_main['date'] = pd.to_datetime(date_main[['year', 'month', 'day']])

# Calculate the day of the week and assign it to a new column
date_main['day_of_week'] = date_main['date'].dt.day_name()
date_main['season'] = date_main['month'].apply(get_season)

date_main = date_main.drop("date", axis=1)
date_main = date_main.drop_duplicates() # drop duplicate dates
date_main = date_main.sort_values(by=['year', 'month', 'day'])

# Generating surrogate keys for the date dataframe and moving the column to the first column of the dataframe

date_main['Surrogate Keys'] = range(1,len(date_main)+1)
date_main = date_main.reindex(columns=['Surrogate Keys'] + list([c for c in date_main.columns if c!= 'Surrogate Keys']))

# Reset and drop the generated (extra) index column
date_main = date_main.reset_index()
date_main = date_main.drop("index",axis=1)
date_main.shape[0]

15492

The other dataframes below are created as filtered versions of the datasets, taking less space in storage while achieving the same purpose. They are then to be merged all together as the final dataset to be used for the loading step.

In [ ]:
# filter pollutant based on dates

# convert to date format
# date_main['Date'] = pd.to_datetime(date_main[['year', 'month', 'day']])
# merge_pollutant_geo['Date'] = merge_pollutant_geo["Date Local"]
# print(merge_pollutant_geo.dtypes)
# merge_pollutant_geo_date = pd.merge(merge_pollutant_geo, date_main[['Date']], on='Date', how='inner')
# # print(merge_pollutant_geo_date.shape[0])
# print(merge_pollutant_geo_date.head())

In [ ]:
date_main.head()

,Surrogate Keys,day,month,year,day_of_week,season
0,1,1,1,1980,Tuesday,Winter
1,2,2,1,1980,Wednesday,Winter
2,3,3,1,1980,Thursday,Winter
3,4,4,1,1980,Friday,Winter
4,5,5,1,1980,Saturday,Winter


In [ ]:
# filter station by geo data
print(station_df.shape[0])
# print(geography_df.head())

filtered_df_test = pd.merge(station_df, geography_df,left_on= ['City','State'] , right_on=['City',"State"], how='left')
print(filtered_df_test.head())
# filter the weather data by location and date
# station_df["Scale Type"] =
print(filtered_df_test.shape[0])



210
   Surrogate Keys_x           ID      Lat       Lon  Status            City  \
0                 1  USC00042863  33.1211 -117.0900  Active       Escondido   
1                 2  USC00166584  31.8142  -93.0856  Active    Natchitoches   
2                 3  USC00280734  40.8292  -75.0833  Active       Belvidere   
3                 4  USC00286055  40.4728  -74.4225  Active  East Brunswick   
4                 5  USC00356749  45.5181 -122.6894  Active        Portland   

        State  Surrogate Keys_y  Area Code  Population  Population Density  \
0  California               NaN        NaN         NaN                 NaN   
1   Louisiana               NaN        NaN         NaN                 NaN   
2  New Jersey               NaN        NaN         NaN                 NaN   
3  New Jersey               NaN        NaN         NaN                 NaN   
4      Oregon             242.0    38900.0   2072553.0              1881.0   

              Timezone  
0                  NaN  
1 

Based on the assumption established in the report, the scaling type measured by the monitoring stations was agreed upon to be decided by the population density of the city in which the station is located in. Given the supporting references, the `National` scaling type is removed since there is no measure of the whole nation (or even global) recoreded or mentioned in the nature of the dataset.
The default value of `Unknown` is given to those records where no location data was able to be extracted, hence these records will ultimately be dropped in the later steps.

In [ ]:
def get_station_type(density):
    # print(density)

    if(density > 0 and density <= 500):
        return "Micro"
    if(density > 501 and density <= 2000):
        return "Middle"
    if(density > 2001 and density <= 4000):
        return "Neighbourhood"
    if(density > 4001 and density <= 6500):
        return "Urban"
    if(density > 6501 and density <= 11000):
        return "Regional"
    return "Unknown"


In [ ]:
station_df["Scale Type"] = filtered_df_test["Population Density"].apply(get_station_type)
station_df.head()

,Surrogate Keys,ID,Lat,Lon,Status,City,State,Scale Type
0,1,USC00042863,33.1211,-117.0900,Active,Escondido,California,Unknown
1,2,USC00166584,31.8142,-93.0856,Active,Natchitoches,Louisiana,Unknown
2,3,USC00280734,40.8292,-75.0833,Active,Belvidere,New Jersey,Unknown
3,4,USC00286055,40.4728,-74.4225,Active,East Brunswick,New Jersey,Unknown
4,5,USC00356749,45.5181,-122.6894,Active,Portland,Oregon,Middle


A measure to provide maximum visibility ranges for users, denoting the range of sight possible correlated with the level of pollution in the area (able to view **up to** the provided value in miles). This is a new measure as part of the FACT table.

In [ ]:
# 1: Hazardous (301-500)
# 1.5: Very Unhealthy (201-300)
# 3: Unhealthy (151-200)
# 5: Unhealthy for Sensitive Groups (101-150)
# 10: Moderate (51-100)
# 20: Good (0-50)
def visibilty_range(type):

    if(type == "Hazardous"):
        return 1
    if(type == "Very Unhealthy"):
        return 1.5
    if(type == "Unhealthy"):
        return 3
    if(type == "Unhealthy for Sensitive Groups"):
        return 5
    if(type == "Moderate"):
        return 10
    if(type == "Good"):
        return 20

In [ ]:
# Creating the FACT table
fact_df = pd.DataFrame()
# Date key
# Geo Key
# Pollutant Key
# weather Key
# station Key

fact_df['Geo Key'] = geography_df['Surrogate Keys']
fact_df['Date Key'] = date_main['Surrogate Keys']
fact_df["Visibility Range"]  = usAQi_df['Category'].apply(visibilty_range)
print(fact_df.head())

   Geo Key  Date Key  Visibility Range
0        1         1              20.0
1        2         2              20.0
2        3         3              20.0
3        4         4              20.0
4        5         5              20.0


In [ ]:
#Outer join
date_main['Date'] = pd.to_datetime(date_main[['year', 'month', 'day']])

weather_df_outerJoin_date= pd.merge(weather_df,date_main,on="Date", how="outer")
weather_df_outerJoin_date.head()

,Surrogate Keys_x,Date,tmax,tmin,prcp,station_id,Surrogate Keys_y,day,month,year,day_of_week,season
0,1.0,1874-01-01,60.0,36.0,0.00,USW00003822,NaN,NaN,NaN,NaN,NaN,NaN
1,186066.0,1874-01-01,44.0,24.0,0.05,USW00093821,NaN,NaN,NaN,NaN,NaN,NaN
2,378978.0,1874-01-01,58.0,37.0,0.00,USW00003017,NaN,NaN,NaN,NaN,NaN,NaN
3,433760.0,1874-01-01,40.0,34.0,0.00,USW00014768,NaN,NaN,NaN,NaN,NaN,NaN
4,585072.0,1874-01-01,41.0,33.0,0.09,USW00093820,NaN,NaN,NaN,NaN,NaN,NaN


Using many inner join commands, all the filtered datasets are merged into one  `.csv` file.

In [ ]:
# date_weather_df = merging date and weather dfs
# date_weather_pollutant_df = merging date_weather and pollutant dfs
# geography_station_df = merging geo and station dfs

# All together df = date_weather_pollutant and geography_station dfs
date_main['Date'] = pd.to_datetime(date_main[['year', 'month', 'day']])

date_weather_df = pd.merge(weather_df,date_main,on="Date", how="inner")
date_weather_df_pollutant = pd.merge(date_weather_df, pollutant_df, on="Date", how="inner")
print(date_weather_df_pollutant.head())

In [ ]:
geography_station_df = pd.merge(geography_df,station_df,on=["State","City"], how="inner")
print(geography_station_df.head())

   Surrogate Keys_x  Area Code         City       State  Population  \
0                 5      10580       Albany    New York      590823   
1                 6      10740  Albuquerque  New Mexico      762853   
2                 7      11100     Amarillo       Texas      202902   
3                15      12020       Athens     Georgia      143081   
4                17      12260      Augusta     Georgia      404125   

   Population Density          Timezone  Surrogate Keys_y           ID  \
0              1747.0  America/New_York                83  USW00014735   
1              1155.0    America/Denver               132  USW00023050   
2               751.0   America/Chicago               131  USW00023047   
3               414.0  America/New_York                52  USW00013873   
4               252.0  America/New_York                13  USW00003820   

       Lat       Lon  Status Scale Type  
0  42.7431  -73.8092  Active     Middle  
1  35.0419 -106.6156  Active     Middle  
2 

Each filtered dataset is saved into a `.csv` file, for easier processing by the kernel.

In [ ]:
# weather_df.to_csv("weather_df.csv", index=False)
# date_main.to_csv("date_main.csv",index=False)
# geography_df.to_csv("geography_df.csv",index=False)
# station_df.to_csv("station_df.csv",index=False)
# print(pollutant_df.head())
# mode_value = pollutant_df['Category'].mode()[0]  # Get the mode of the 'Category' column
# pollutant_df['Category'] = pollutant_df['Category'].fillna(mode_value)
# print(pollutant_df.head())
# pollutant_df = pollutant_df.reset_index()
# pollutant_df = pollutant_df.drop("index",axis=1)
print(pollutant_df.head())
pollutant_df.to_csv("pollutant_df.csv")

New dataframes are created, based upon the previously transformed datasets.

In [ ]:
csv_station = pd.read_csv('station_df.csv')
csv_pollutant = pd.read_csv('pollutant_df.csv')
csv_weather = pd.read_csv('weather_df.csv')
csv_geography = pd.read_csv('geography_df.csv')
csv_date = pd.read_csv('date_main.csv')

And now the merging commands begin; many different pairings of the datasets were merged previously (refer to the `Merging.ipynb` file for different trials). However, the results increased the number of rows by a vast number, not suitable for the final merging step (even not capable by the kernel ude to its limited hardware power). To avoid using hardware augmentation, other manners were sought out.

In [ ]:
# weather and station dataframes based on the station id

weather_station_df = pd.merge(csv_weather, csv_station, left_on=['station_id'], right_on=['ID'], how="inner")
print(weather_station_df.head(0))
weather_station_df = weather_station_df.drop(["Surrogate Keys_y","ID", 'Lat', 'Lon'],axis=1)
print(weather_station_df.head())
print(weather_station_df.shape[0])

Empty DataFrame
Columns: [Surrogate Keys_x, Date, tmax, tmin, prcp, station_id, Surrogate Keys_y, ID, Lat, Lon, Status, City, State, Scale Type]
Index: []
   Surrogate Keys_x        Date  tmax  tmin  prcp   station_id  Status  \
0                 1  1874-01-01    60    36  0.00  USW00003822  Active   
1                 2  1874-01-02    62    43  0.02  USW00003822  Active   
2                 3  1874-01-03    66    53  0.12  USW00003822  Active   
3                 4  1874-01-04    70    59  0.66  USW00003822  Active   
4                 5  1874-01-05    66    58  0.50  USW00003822  Active   

       City    State Scale Type  
0  Savannah  Georgia     Middle  
1  Savannah  Georgia     Middle  
2  Savannah  Georgia     Middle  
3  Savannah  Georgia     Middle  
4  Savannah  Georgia     Middle  
9757061


In [ ]:
# date and weather_station dataframes based on the date
date_weather_station_df = pd.merge(csv_date, weather_station_df, on="Date", how="inner")
print(date_weather_station_df.head(0))
date_weather_station_df = date_weather_station_df.drop(["Surrogate Keys_x"],axis=1)
print(date_weather_station_df.head())
print(date_weather_station_df.shape[0])
date_weather_station_df.to_csv('date_weather_station_df.csv', index=False)

Empty DataFrame
Columns: [Surrogate Keys, day, month, year, day_of_week, season, Date, Surrogate Keys_x, tmax, tmin, prcp, station_id, Status, City, State, Scale Type]
Index: []
   Surrogate Keys  day  month  year day_of_week  season        Date  tmax  \
0               1    1      1  1980     Tuesday  Winter  1980-01-01    50   
1               1    1      1  1980     Tuesday  Winter  1980-01-01    58   
2               1    1      1  1980     Tuesday  Winter  1980-01-01    47   
3               1    1      1  1980     Tuesday  Winter  1980-01-01    28   
4               1    1      1  1980     Tuesday  Winter  1980-01-01    35   

   tmin  prcp   station_id  Status            City        State Scale Type  
0    42   0.0  USW00003822  Active        Savannah      Georgia     Middle  
1    52   0.0  USW00024286  Active   Crescent City   California     Middle  
2    37   0.0  USW00013833  Active     Hattiesburg  Mississippi      Micro  
3    22   0.0  USW00023066  Active  Grand Junction 

In [ ]:
# geography and pollutant dataframes based on the city and state
geography_pollutant_df = pd.merge( csv_pollutant, csv_geography, on=['State','City'], how="inner")
print(geography_pollutant_df.head(0))
geography_pollutant_df = geography_pollutant_df.drop(["Unnamed: 0","Surrogate Keys_y"],axis=1)
print(geography_pollutant_df.head())
print(geography_pollutant_df.shape[0])
geography_pollutant_df.to_csv('geography_pollutant_df.csv', index=False)

Empty DataFrame
Columns: [Unnamed: 0, Surrogate Keys_x, Date, State, City, NO2 Mean, CO Mean, SO2 Mean, O3 Mean, PM2.5 Mean, PM10 Mean, Category, Surrogate Keys_y, Area Code, Population, Population Density, Timezone]
Index: []
   Surrogate Keys_x        Date     State    City  NO2 Mean   CO Mean  \
0                 1  1980-01-01  Illinois  Peoria  0.000000  0.462500   
1               688  1980-01-02  Illinois  Peoria  0.030000  0.858333   
2              1651  1980-01-03  Illinois  Peoria  0.066957  0.770000   
3              1897  1980-01-04  Illinois  Peoria  0.038636  1.266667   
4              2420  1980-01-05  Illinois  Peoria  0.000000  1.137500   

   SO2 Mean   O3 Mean  PM2.5 Mean  PM10 Mean Category  Area Code  Population  \
0  0.000375  0.008706    0.015688      0.024     Good      37900      253461   
1  0.001208  0.002529    0.015688      0.024     Good      37900      253461   
2  0.006987  0.004235    0.015688      0.024     Good      37900      253461   
3  0.010042  0

The `final_df` dataframe as a result of the `Merging.ipynb` file, where the new measures are appended, based upon calculated attributes, and extracted attributes of the main dataset.

In [ ]:
final_df = pd.read_csv("final_df.csv")

In [ ]:
final_df.head()
final_df["Visibility Range"]  = final_df['Category'].apply(visibilty_range)

In [ ]:
final_df['Date'] = pd.to_datetime(final_df[['year', 'month', 'day']])
final_df.head()


,Surrogate Keys,day,month,year,season,day_of_week,City,State,Population,Population Density,...,Status,NO2 Mean,CO Mean,SO2 Mean,O3 Mean,PM2.5 Mean,PM10 Mean,Category,Visibility Range,Date
0,1,1,1,1980,Winter,Tuesday,Dallas,Texas,5910669,1522.0,...,Active,0.000000,1.206250,0.002083,0.020117,0.015688,0.024,Moderate,10.0,1980-01-01
1,2,2,1,1980,Winter,Wednesday,Dallas,Texas,5910669,1522.0,...,Active,0.030000,1.571667,0.001500,0.011064,0.015688,0.024,Moderate,10.0,1980-01-02
2,3,3,1,1980,Winter,Thursday,Dallas,Texas,5910669,1522.0,...,Active,0.066957,1.549185,0.000000,0.008588,0.015688,0.024,Good,20.0,1980-01-03
3,4,4,1,1980,Winter,Friday,Dallas,Texas,5910669,1522.0,...,Active,0.038636,1.038095,0.000000,0.007403,0.015688,0.024,Good,20.0,1980-01-04
4,5,5,1,1980,Winter,Saturday,Dallas,Texas,5910669,1522.0,...,Active,0.000000,0.637500,0.000000,0.006442,0.015688,0.024,Good,20.0,1980-01-05


In [ ]:
# The AQI values of the main dataset are merged with the final_df DataFrame based on the 'Date', 'State', and 'City' columns
final_df_test = pd.merge(final_df,usAQi_df,left_on=['Date',"State","City"], right_on=['Date',"state_name","city_ascii"], how="inner")
final_df_test['AQI']
# The unnecessary columns are dropped
final_df = final_df_test.drop(["Defining Parameter","Number of Sites Reporting","city_ascii","state_id",'state_name','lat','lng','population','density',"timezone"],axis=1)

In [ ]:
print(final_df.columns)

Index(['Surrogate Keys', 'day', 'month', 'year', 'season', 'day_of_week',
       'City', 'State', 'Population', 'Population Density', 'Timezone',
       'Area Code', 'tmax', 'tmin', 'prcp', 'ID', 'Scale Type', 'Status',
       'NO2 Mean', 'CO Mean', 'SO2 Mean', 'O3 Mean', 'PM2.5 Mean', 'PM10 Mean',
       'Category_x', 'Visibility Range', 'Date', 'Unnamed: 0', 'CBSA Code',
       'AQI', 'Category_y'],
      dtype='object')


In [ ]:
final_df = final_df.drop(['CBSA Code','Unnamed: 0',"Category_y"],axis=1)


This method is for obtaining the AQHI (air quality health index) values based on the AQI (air quality index) values. AQI communicates the air quality of the single worst pollutant. The index rating for the new AQHI is the sum of the health risks from each of the pollutants in the index.

In [ ]:
def getAQHi(n):
    if(n >= 0 and n <= 50):
        return 0
    if(n >= 51 and n <= 100):
        return 1
    if(n >= 101 and n <= 150):
        return 2
    else:
        return 3

In [ ]:
final_df["AQHI"]  = final_df['AQI'].apply(getAQHi)

In [ ]:
final_df = final_df.rename(columns={'Category_x':"Category"})
final_df = final_df.drop("Date",axis=1)
final_df.head()

,Surrogate Keys,day,month,year,season,day_of_week,City,State,Population,Population Density,...,NO2 Mean,CO Mean,SO2 Mean,O3 Mean,PM2.5 Mean,PM10 Mean,Category,Visibility Range,AQI,AQHI
0,1,1,1,1980,Winter,Tuesday,Dallas,Texas,5910669,1522.0,...,0.000000,1.206250,0.002083,0.020117,0.015688,0.024,Moderate,10.0,51,1
1,2,2,1,1980,Winter,Wednesday,Dallas,Texas,5910669,1522.0,...,0.030000,1.571667,0.001500,0.011064,0.015688,0.024,Moderate,10.0,57,1
2,3,3,1,1980,Winter,Thursday,Dallas,Texas,5910669,1522.0,...,0.066957,1.549185,0.000000,0.008588,0.015688,0.024,Good,20.0,36,0
3,4,4,1,1980,Winter,Friday,Dallas,Texas,5910669,1522.0,...,0.038636,1.038095,0.000000,0.007403,0.015688,0.024,Good,20.0,38,0
4,5,5,1,1980,Winter,Saturday,Dallas,Texas,5910669,1522.0,...,0.000000,0.637500,0.000000,0.006442,0.015688,0.024,Good,20.0,38,0


In [ ]:
final_df.columns

Index(['Surrogate Keys', 'day', 'month', 'year', 'season', 'day_of_week',
       'City', 'State', 'Population', 'Population Density', 'Timezone',
       'Area Code', 'tmax', 'tmin', 'prcp', 'ID', 'Scale Type', 'Status',
       'NO2 Mean', 'CO Mean', 'SO2 Mean', 'O3 Mean', 'PM2.5 Mean', 'PM10 Mean',
       'Category', 'Visibility Range', 'AQI', 'AQHI'],
      dtype='object')

In [ ]:
final_df.to_csv("Data_Staging.csv",index=False)

The final dataframe `final_df` is now converted into its `.csv` couterpart (`Data_Staging.csv`) to be used to load in the database instance.